# 1. UK HC

In [ ]:
import pandas as pd
import json
import copy

In [ ]:
df = pd.read_csv("int_data/UK_extracted_hc_multilingual.csv")


In [ ]:
df

prompt generation

In [ ]:
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
# Make sure df is already defined (e.g. via pandas.read_csv or similar)
# from openai import OpenAI  <-- imported inside processing functions as needed

# ------------------------------------------------------------------------------
# Dictionaries for language-specific text (as in your original code)
# ------------------------------------------------------------------------------

# -------------------------------------------------------------------------------
# Updated system_instructions: if claim is supported by evidence, answer "1"; if contradicted, answer "0".
# If unsure, choose the option closest to the evidence.
# ------------------------------------------------------------------------------
system_instructions = {
    "English": (
        "You are an expert in evaluating medical claims. Your task is to assess the truthfulness of the provided health claim "
        "using established medical research, guidelines, and evidence. Determine if the claim is true. If the claim is supported by evidence, "
        "respond with '1'. If the claim is contradicted by evidence, respond with '0'. If you are unsure, choose the option that is closest to the available evidence. "
        "Additional context is provided below: the 'Nutrient substance, food or food category' indicates the food or supplement involved, and the 'Health relationship' describes "
        "the intended health effect."
    ),
    "Spanish": (
        "Eres un experto en la evaluación de afirmaciones médicas. Tu tarea es evaluar la veracidad de la afirmación de salud proporcionada "
        "utilizando investigaciones médicas establecidas, directrices y evidencias. Determina si la afirmación es verdadera. Si la afirmación está respaldada por evidencia, "
        "responde con '1'. Si la afirmación contradice la evidencia, responde con '0'. Si no estás seguro, elige la opción que más se aproxime a la evidencia disponible. "
        "Se proporciona contexto adicional a continuación: 'Sustancia nutritiva, alimento o categoría de alimentos' indica el alimento o suplemento involucrado, y 'Relación de salud' describe "
        "el efecto de salud previsto."
    ),
    "Russian": (
        "Вы являетесь экспертом в оценке медицинских заявлений. Ваша задача — оценить достоверность предоставленного заявления о здоровье, "
        "используя установленные медицинские исследования, рекомендации и доказательства. Определите, является ли заявление правдивым. Если заявление подтверждено доказательствами, "
        "ответьте '1'. Если заявление противоречит доказательствам, ответьте '0'. Если вы не уверены, выберите вариант, который наиболее близок к имеющимся доказательствам. "
        "Дополнительный контекст приведён ниже: «Пищевая добавка, продукт или категория продуктов» указывает на используемый продукт или пищевую добавку, а «Связь со здоровьем» описывает "
        "предполагаемый эффект для здоровья."
    ),
    "German": (
        "Sie sind ein Experte für die Bewertung medizinischer Aussagen. Ihre Aufgabe ist es, die Wahrhaftigkeit der vorgelegten Gesundheitsaussage "
        "mithilfe etablierter medizinischer Forschung, Richtlinien und Beweise zu beurteilen. Bestimmen Sie, ob die Aussage wahr ist. Wenn die Aussage durch Beweise gestützt wird, "
        "antworten Sie mit '1'. Wenn die Aussage den Beweisen widerspricht, antworten Sie mit '0'. Falls Sie unsicher sind, wählen Sie bitte die Option, die den verfügbaren Beweisen am nächsten kommt. "
        "Weiterer Kontext wird unten bereitgestellt: 'Nährstoffsubstanz, Lebensmittel oder Lebensmittelkategorie' gibt an, welches Lebensmittel oder Nahrungsergänzungsmittel beteiligt ist, und "
        "'Gesundheitsbeziehung' beschreibt den beabsichtigten gesundheitlichen Effekt."
    ),
    "French": (
        "Vous êtes un expert dans l'évaluation des allégations médicales. Votre tâche consiste à évaluer la véracité de l'allégation de santé fournie "
        "en utilisant des recherches médicales établies, des directives et des preuves. Déterminez si l'allégation est vraie. Si l'allégation est étayée par des preuves, "
        "répondez par '1'. Si l'allégation contredit les preuves, répondez par '0'. Si vous n'êtes pas sûr, choisissez l'option qui se rapproche le plus des preuves disponibles. "
        "Un contexte supplémentaire est fourni ci-dessous : 'Substance nutritive, aliment ou catégorie d'aliments' indique l'aliment ou le complément impliqué, et 'Relation de santé' décrit "
        "l'effet de santé envisagé."
    ),
    "Japanese": (
        "あなたは医療クレームの評価に精通した専門家です。あなたの任務は、確立された医学的研究、ガイドライン、証拠を用いて、提供された健康クレームの真実性を評価することです。 "
        "クレームが真実であるかを判断してください。もし証拠に裏付けられている場合は『1』と回答し、証拠と矛盾する場合は『0』と回答してください。不明な場合は、利用可能な証拠に最も近い選択肢を選んでください。 "
        "以下に追加のコンテキストが提供されています：『栄養素、食品または食品カテゴリー』は関係する食品またはサプリメントを示し、『健康関係』は意図された健康効果を説明します。"
    ),
    "Portuguese": (
        "Você é um especialista na avaliação de reivindicações médicas. Sua tarefa é avaliar a veracidade da reivindicação de saúde fornecida "
        "utilizando pesquisas médicas estabelecidas, diretrizes e evidências. Determine se a reivindicação é verdadeira. Se a reivindicação for suportada por evidências, "
        "responda com '1'. Se a reivindicação contradiz as evidências, responda com '0'. Se estiver em dúvida, escolha a opção que mais se aproxima das evidências disponíveis. "
        "Contexto adicional é fornecido abaixo: 'Substância nutritiva, alimento ou categoria de alimentos' indica o alimento ou suplemento envolvido, e 'Relação de saúde' descreve "
        "o efeito de saúde pretendido."
    ),
    "Turkish": (
        "Tıbbi iddiaları değerlendirme konusunda uzmansınız. Göreviniz, yerleşik tıbbi araştırmalar, yönergeler ve kanıtlar kullanarak "
        "sağlanan sağlık iddiasının doğruluğunu değerlendirmektir. İddianın doğru olup olmadığını belirleyin. İddia kanıtlarla destekleniyorsa, '1' ile cevap verin. "
        "İddia kanıtlara ters düşüyorsa, '0' ile cevap verin. Emin değilseniz, mevcut kanıtlara en yakın olan seçeneği tercih edin. Aşağıda ek bağlam sağlanmıştır: "
        "'Besin maddesi, gıda veya gıda kategorisi' ilgili gıda veya takviyeyi belirtir ve 'Sağlık ilişkisi' amaçlanan sağlık etkisini açıklar."
    ),
    "Italian": (
        "Sei un esperto nella valutazione delle affermazioni mediche. Il tuo compito è valutare la veridicità dell'affermazione sulla salute fornita "
        "utilizzando ricerche mediche consolidate, linee guida ed evidenze. Determina se l'affermazione è vera. Se l'affermazione è supportata dalle evidenze, rispondi con '1'. "
        "Se l'affermazione contraddice le evidenze, rispondi con '0'. Se non sei sicuro, scegli l'opzione che più si avvicina alle evidenze disponibili. "
        "Ulteriori contesti sono forniti di seguito: 'Sostanza nutritiva, alimento o categoria di alimenti' indica l'alimento o il supplemento coinvolto, e 'Relazione di salute' descrive "
        "l'effetto previsto sulla salute."
    ),
    "Persian": (
        "شما یک متخصص در ارزیابی ادعاهای پزشکی هستید. وظیفه شما این است که صحت ادعای ارائه شده در زمینه سلامت را با استفاده از تحقیقات پزشکی معتبر، "
        "دستورالعمل‌ها و شواهد موجود ارزیابی کنید. تعیین کنید که آیا ادعا صحیح است. اگر ادعا با شواهد تأیید شده است، با '1' پاسخ دهید. اگر ادعا با شواهد مغایرت دارد، با '0' پاسخ دهید. "
        "اگر مطمئن نیستید، نزدیک‌ترین گزینه به شواهد موجود را انتخاب کنید. زمینه اضافی در زیر ارائه شده است: 'ماده مغذی، غذا یا دسته‌بندی غذا' نشان‌دهنده غذایی است که در آن ادعا مطرح شده و 'رابطه سلامت' اثر مورد نظر بر سلامت را توضیح می‌دهد."
    ),
    "Dutch": (
        "U bent een expert in het evalueren van medische claims. Uw taak is om de waarheidsgetrouwheid van de verstrekte gezondheidsclaim te beoordelen "
        "met behulp van gevestigde medische onderzoeken, richtlijnen en bewijzen. Bepaal of de claim waar is. Als de claim door bewijzen wordt ondersteund, "
        "antwoord dan met '1'. Als de claim in tegenspraak is met de bewijzen, antwoord dan met '0'. Als u niet zeker bent, kiest u de optie die het dichtst bij het beschikbare bewijs ligt. "
        "Verdere context wordt hieronder gegeven: 'Voedingsstof, voeding of voedselcategorie' geeft aan om welk voedsel of supplement het gaat, en 'Gezondheidsrelatie' beschrijft "
        "het beoogde gezondheidseffect."
    ),
    "Polish": (
        "Jesteś ekspertem w ocenie twierdzeń medycznych. Twoim zadaniem jest ocenić prawdziwość podanego twierdzenia dotyczącego zdrowia, "
        "korzystając z uznanych badań medycznych, wytycznych i dowodów. Określ, czy twierdzenie jest prawdziwe. Jeśli twierdzenie jest poparte dowodami, odpowiedz '1'. "
        "Jeśli twierdzenie przeczy dowodom, odpowiedz '0'. Jeśli nie jesteś pewien, wybierz opcję najbliższą dostępnym dowodom. Dodatkowy kontekst podany jest poniżej: "
        "'Substancja odżywcza, żywność lub kategoria żywności' wskazuje na dany produkt lub suplement, a 'Relacja zdrowotna' opisuje zamierzony efekt zdrowotny."
    ),
    "Chinese": (
        "您是评估医疗声明的专家。您的任务是利用既定的医学研究、指南和证据来评估所提供健康声明的真实性。请判断该声明是否真实。"
        "如果声明得到证据支持，请回答 '1'；如果声明与证据相悖，请回答 '0'；如果不确定，请选择最接近现有证据的选项。"
        "下面提供了额外的上下文：“营养物质、食品或食品类别”指的是所涉及的食品或补充剂，而“健康关系”描述了预期的健康效果。"
    ),
    "Vietnamese": (
        "Bạn là một chuyên gia trong việc đánh giá các tuyên bố y tế. Nhiệm vụ của bạn là đánh giá tính chân thực của tuyên bố về sức khỏe được cung cấp, "
        "dựa trên các nghiên cứu y tế đã được thiết lập, các hướng dẫn và bằng chứng. Hãy xác định xem tuyên bố đó có thật không. Nếu tuyên bố được hỗ trợ bởi bằng chứng, hãy trả lời bằng '1'. "
        "Nếu tuyên bố mâu thuẫn với bằng chứng, hãy trả lời bằng '0'. Nếu không chắc chắn, hãy chọn phương án gần với bằng chứng nhất. "
        "Bối cảnh bổ sung được cung cấp bên dưới: ‘Chất dinh dưỡng, thực phẩm hoặc loại thực phẩm’ chỉ ra sản phẩm hoặc thực phẩm bổ sung liên quan, và ‘Mối quan hệ sức khỏe’ mô tả hiệu ứng sức khỏe dự kiến."
    ),
    "Indonesian": (
        "Anda adalah seorang ahli dalam mengevaluasi klaim medis. Tugas Anda adalah menilai kebenaran klaim kesehatan yang diberikan dengan menggunakan penelitian medis yang mapan, "
        "pedoman, dan bukti. Tentukan apakah klaim tersebut benar. Jika klaim didukung oleh bukti, jawablah dengan '1'. Jika klaim bertentangan dengan bukti, jawablah dengan '0'. "
        "Jika Anda tidak yakin, pilihlah opsi yang paling mendekati bukti yang ada. Konteks tambahan disediakan di bawah ini: 'Zat gizi, makanan atau kategori makanan' menunjukkan makanan atau suplemen yang terlibat, "
        "dan 'Hubungan kesehatan' menggambarkan efek kesehatan yang dimaksudkan."
    ),
    "Czech": (
        "Jste odborník na hodnocení lékařských tvrzení. Vaším úkolem je posoudit pravdivost poskytnutého zdravotního tvrzení pomocí zavedeného lékařského výzkumu, pokynů a důkazů. "
        "Určete, zda je tvrzení pravdivé. Pokud je tvrzení podpořeno důkazy, odpovězte '1'. Pokud tvrzení odporuje důkazům, odpovězte '0'. "
        "Pokud si nejste jisti, vyberte možnost, která se nejvíce blíží dostupným důkazům. Níže je uveden další kontext: 'Výživová látka, potravina nebo kategorie potravin' označuje zapojenou potravinu nebo doplněk stravy a "
        "'Zdravotní vztah' popisuje zamýšlený zdravotní účinek."
    ),
    "Korean": (
        "당신은 의료 주장 평가의 전문가입니다. 확립된 의학적 연구, 지침 및 증거를 활용하여 제공된 건강 주장의 진실성을 평가하십시오. "
        "주장이 증거로 뒷받침되면 '1'로, 증거와 상반되면 '0'로 답하십시오. 만약 확신이 없다면, 가능한 증거에 가장 근접한 선택지를 고르십시오. "
        "아래에 추가 문맥이 제공됩니다: '영양소, 식품 또는 식품 범주'는 관련 식품이나 보충제를 나타내며, '건강 관계'는 의도된 건강 효과를 설명합니다."
    ),
    "Ukrainian": (
        "Ви є експертом з оцінки медичних заяв. Ваше завдання — оцінити достовірність наданої заяви про здоров'я, використовуючи встановлені медичні дослідження, рекомендації та докази. "
        "Визначте, чи є заява правдивою. Якщо заява підтверджена доказами, відповідайте '1'. Якщо заява суперечить доказам, відповідайте '0'. "
        "Якщо ви не впевнені, оберіть варіант, який найбільше відповідає наявним доказам. Додатковий контекст наведено нижче: 'Харчова добавка, їжа або категорія їжі' вказує на залучений продукт або добавку, а "
        "'Зв'язок зі здоров'ям' описує передбачуваний вплив на здоров'я."
    ),
    "Arabic": (
        "أنت خبير في تقييم المطالبات الطبية. مهمتك هي تقييم صحة المطالبة الصحية المقدمة باستخدام البحوث الطبية المعتمدة والإرشادات والأدلة. "
        "حدد ما إذا كانت المطالبة صحيحة. إذا كانت المطالبة مدعومة بالأدلة، فقم بالرد بـ '1'. إذا كانت المطالبة تتناقض مع الأدلة، فقم بالرد بـ '0'. "
        "إذا لم تكن متأكدًا، اختر الخيار الأقرب إلى الأدلة المتوفرة. يتم تقديم سياق إضافي أدناه: 'المادة الغذائية، الطعام أو فئة الطعام' تشير إلى الطعام أو المكمل المعني، و"
        "'العلاقة الصحية' تصف التأثير الصحي المقصود."
    ),
    "Greek": (
        "Είστε ειδικός στην αξιολόγηση ιατρικών ισχυρισμών. Το έργο σας είναι να αξιολογήσετε την αλήθεια του παρεχόμενου ισχυρισμού υγείας, χρησιμοποιώντας καθιερωμένη ιατρική έρευνα, "
        "κατευθυντήριες οδηγίες και αποδείξεις. Καθορίστε αν ο ισχυρισμός είναι αληθής. Εάν ο ισχυρισμός υποστηρίζεται από αποδείξεις, απαντήστε με '1'. Εάν ο ισχυρισμός έρχεται σε αντίθεση με τις αποδείξεις, απαντήστε με '0'. "
        "Εάν δεν είστε βέβαιοι, επιλέξτε την επιλογή που πλησιάζει περισσότερο στις διαθέσιμες αποδείξεις. Επιπλέον πλαίσιο παρέχεται παρακάτω: 'Θρεπτική ουσία, τρόφιμο ή κατηγορία τροφίμων' υποδεικνύει το εμπλεκόμενο τρόφιμο ή συμπλήρωμα, και "
        "'Σχέση υγείας' περιγράφει το επιδιωκόμενο αποτέλεσμα."
    ),
    "Hindi": (
        "आप चिकित्सा दावों का मूल्यांकन करने में विशेषज्ञ हैं। आपका कार्य प्रदान किए गए स्वास्थ्य दावे की सत्यता का आकलन करना है, इसके लिए स्थापित चिकित्सा अनुसंधान, दिशानिर्देश और सबूतों का उपयोग करें। "
        "निर्धारित करें कि दावा सही है। यदि दावा सबूतों द्वारा समर्थित है, तो '1' के साथ उत्तर दें। यदि दावा सबूतों के विपरीत है, तो '0' के साथ उत्तर दें। "
        "यदि आप सुनिश्चित नहीं हैं, तो उपलब्ध सबूतों के अनुसार सबसे उपयुक्त विकल्प चुनें। अतिरिक्त संदर्भ नीचे प्रदान किया गया है: 'पोषक तत्व, खाद्य या खाद्य वर्ग' से तात्पर्य उस खाद्य या अनुपूरक से है जो शामिल है, और "
        "'स्वास्थ्य संबंध' से आशय लक्षित स्वास्थ्य प्रभाव से है।"
    )
}

# ------------------------------------------------------------------------------
# Updated strict_response_lines: only "1" or "0" are allowed, with no extra text.
# ------------------------------------------------------------------------------
strict_response_lines = {
    "English": "Your answer must be exactly one of the following: '1' or '0'. Do not include any additional text.",
    "Spanish": "Su respuesta debe ser exactamente una de las siguientes: '1' o '0'. No incluya ningún texto adicional.",
    "Russian": "Ваш ответ должен быть ровно одним из следующих: '1' или '0'. Не добавляйте никакого дополнительного текста.",
    "German": "Ihre Antwort muss genau eine der folgenden Optionen sein: '1' oder '0'. Fügen Sie keinen zusätzlichen Text hinzu.",
    "French": "Votre réponse doit être exactement l'une des options suivantes : '1' ou '0'. N'incluez aucun texte supplémentaire.",
    "Japanese": "あなたの回答は次のいずれか一つに正確にしてください：『1』または『0』。追加のテキストを含めないでください。",
    "Portuguese": "Sua resposta deve ser exatamente uma das seguintes: '1' ou '0'. Não inclua nenhum texto adicional.",
    "Turkish": "Cevabınız tam olarak aşağıdakilerden biri olmalıdır: '1' veya '0'. Ekstra metin eklemeyin.",
    "Italian": "La tua risposta deve essere esattamente una delle seguenti: '1' o '0'. Non includere testo aggiuntivo.",
    "Persian": "پاسخ شما باید دقیقا یکی از گزینه‌های زیر باشد: '1' یا '0'. هیچ متن اضافی را اضافه نکنید.",
    "Dutch": "Uw antwoord moet exact een van de volgende zijn: '1' of '0'. Voeg geen extra tekst toe.",
    "Polish": "Twoja odpowiedź musi być dokładnie jedną z następujących: '1' lub '0'. Nie dodawaj żadnego dodatkowego tekstu.",
    "Chinese": "您的回答必须严格为以下其中之一：‘1’ 或 ‘0’。请不要包含任何额外的文字。",
    "Vietnamese": "Câu trả lời của bạn phải chính xác là một trong các lựa chọn sau: '1' hoặc '0'. Không thêm bất kỳ văn bản bổ sung nào.",
    "Indonesian": "Jawaban Anda harus tepat salah satu dari yang berikut: '1' atau '0'. Jangan sertakan teks tambahan apa pun.",
    "Czech": "Vaše odpověď musí být přesně jedna z následujících: '1' nebo '0'. Nezahrnujte žádný další text.",
    "Korean": "귀하의 답변은 반드시 다음 중 하나여야 합니다: '1' 또는 '0'. 추가 텍스트를 포함하지 마십시오.",
    "Ukrainian": "Ваша відповідь має бути точно однією з наступних: '1' або '0'. Не додавайте ніякого додаткового тексту.",
    "Arabic": "يجب أن يكون جوابك هو أحد الخيارات التالية بالضبط: '1' أو '0'. لا تقم بإضافة أي نص إضافي.",
    "Greek": "Η απάντησή σας πρέπει να είναι ακριβώς μία από τις εξής: '1' ή '0'. Μην προσθέτετε επιπλέον κείμενο.",
    "Hindi": "आपका उत्तर ठीक उसी में से एक होना चाहिए: '1' या '0'. किसी भी अतिरिक्त पाठ को शामिल न करें।"
}

# ------------------------------------------------------------------------------
# Prompt labels for the claim details in each language
# ------------------------------------------------------------------------------
prompt_labels = {
    "English": {
        "claim": "Claim:",
        "nutrient": "Nutrient substance, food or food category:",
        "health": "Health relationship:"
    },
    "Spanish": {
        "claim": "Afirmación:",
        "nutrient": "Sustancia nutritiva, alimento o categoría de alimentos:",
        "health": "Relación de salud:"
    },
    "Russian": {
        "claim": "Заявление:",
        "nutrient": "Пищевая добавка, продукт или категория продуктов:",
        "health": "Связь со здоровьем:"
    },
    "German": {
        "claim": "Behauptung:",
        "nutrient": "Nährstoffsubstanz, Lebensmittel oder Lebensmittelkategorie:",
        "health": "Gesundheitsbeziehung:"
    },
    "French": {
        "claim": "Allégation:",
        "nutrient": "Substance nutritive, aliment ou catégorie d'aliments:",
        "health": "Relation de santé:"
    },
    "Japanese": {
        "claim": "クレーム:",
        "nutrient": "栄養素、食品または食品カテゴリー:",
        "health": "健康関係:"
    },
    "Portuguese": {
        "claim": "Alegação:",
        "nutrient": "Substância nutritiva, alimento ou categoria de alimentos:",
        "health": "Relação de saúde:"
    },
    "Turkish": {
        "claim": "İddia:",
        "nutrient": "Besin maddesi, gıda veya gıda kategorisi:",
        "health": "Sağlık ilişkisi:"
    },
    "Italian": {
        "claim": "Affermazione:",
        "nutrient": "Sostanza nutritiva, alimento o categoria di alimenti:",
        "health": "Relazione di salute:"
    },
    "Persian": {
        "claim": "ادعا:",
        "nutrient": "ماده مغذی، غذا یا دسته‌بندی غذا:",
        "health": "رابطه سلامت:"
    },
    "Dutch": {
        "claim": "Claim:",
        "nutrient": "Voedingsstof, voeding of voedselcategorie:",
        "health": "Gezondheidsrelatie:"
    },
    "Polish": {
        "claim": "Twierdzenie:",
        "nutrient": "Substancja odżywcza, żywność lub kategoria żywności:",
        "health": "Relacja zdrowotna:"
    },
    "Chinese": {
        "claim": "声明:",
        "nutrient": "营养物质、食品或食品类别:",
        "health": "健康关系:"
    },
    "Vietnamese": {
        "claim": "Tuyên bố:",
        "nutrient": "Chất dinh dưỡng, thực phẩm hoặc loại thực phẩm:",
        "health": "Mối quan hệ sức khỏe:"
    },
    "Indonesian": {
        "claim": "Klaim:",
        "nutrient": "Zat gizi, makanan atau kategori makanan:",
        "health": "Hubungan kesehatan:"
    },
    "Czech": {
        "claim": "Tvrzení:",
        "nutrient": "Výživová látka, potravina nebo kategorie potravin:",
        "health": "Zdravotní vztah:"
    },
    "Korean": {
        "claim": "주장:",
        "nutrient": "영양소, 식품 또는 식품 범주:",
        "health": "건강 관계:"
    },
    "Ukrainian": {
        "claim": "Заява:",
        "nutrient": "Харчова добавка, їжа або категорія їжі:",
        "health": "Зв'язок зі здоров'ям:"
    },
    "Arabic": {
        "claim": "المطالبة:",
        "nutrient": "المادة الغذائية، الطعام أو فئة الطعام:",
        "health": "العلاقة الصحية:"
    },
    "Greek": {
        "claim": "Ισχυρισμός:",
        "nutrient": "Θρεπτική ουσία, τρόφιμο ή κατηγορία τροφίμων:",
        "health": "Σχέση υγείας:"
    },
    "Hindi": {
        "claim": "दावा:",
        "nutrient": "पोषक तत्व, खाद्य या खाद्य वर्ग:",
        "health": "स्वास्थ्य संबंध:"
    }
}

# ------------------------------------------------------------------------------
# Function to create a combined prompt entry for a given claim in a specific language,
# including model and iteration information.
# ------------------------------------------------------------------------------
def create_prompt_entry(claim_id, translation_claim, translation_nutrient_substance, translation_health_relationship, language, model, iteration):
    # Get language-specific texts; default to English if not available
    sys_instr = system_instructions.get(language, system_instructions["English"])
    labels = prompt_labels.get(language, prompt_labels["English"])
    strict_line = strict_response_lines.get(language, strict_response_lines["English"])
    
    # Build the combined prompt:
    # 1. The system instruction in the target language
    # 2. The claim details using the target language's prompt labels
    # 3. The strict instruction in the target language
    prompt_text = (
        f"{sys_instr}\n\n"
        f"{labels['claim']} {translation_claim}\n"
        f"{labels['nutrient']} {translation_nutrient_substance}\n"
        f"{labels['health']} {translation_health_relationship}\n\n"
        f"{strict_line}"
    )
    
    # Create a custom ID that includes the claim ID, language, model, and iteration
    custom_id = f"claim_{claim_id}_{language}__{model}__{iteration}"
    
    return {
        "custom_id": custom_id,
        "prompt": prompt_text
    }

# ------------------------------------------------------------------------------
# Build the list of prompt entries across models and iterations
# ------------------------------------------------------------------------------
prompts = []
models = [
    "gpt-4o-mini-2024-07-18"
]
iterations = range(3)  # e.g., 0, 1, 2

for model in models:
    for _, row in df.iterrows():
        #print(row)
        language = row['language']  # e.g., "English", "Spanish", etc.
        for iteration in iterations:
            entry = create_prompt_entry(
                row['claim_id'],
                row['translation_claim'],
                row['translation_nutrient_substance'],
                row['translation_health_relationship'],
                language,
                model,
                iteration
            )
            prompts.append(entry)


In [ ]:
len(prompts)

In [ ]:
import os
import json
from tqdm import tqdm
from openai import OpenAI
os.environ["OPENAI_API_KEY"]="KEY" 

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
import os, json
from tqdm import tqdm
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed


def call_openai(entry):
    custom_id = entry["custom_id"]
    prompt    = entry["prompt"]
    model     = custom_id.split("__")[1]
    try:
        resp = client.chat.completions.create(
            model      = model,
            messages   = [{"role":"user","content":prompt}],
            temperature=0.7,
            max_tokens =50
        )
        return {
            "custom_id": custom_id,
            "model":     model,
            "response":  resp.choices[0].message.content.strip()
        }
    except Exception as e:
        return {
            "custom_id": custom_id,
            "model":     model,
            "error":     str(e)
        }

responses = []
# choose a worker count based on your bandwidth & rate‐limit headroom
with ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(call_openai, p) for p in prompts]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Parallelized"):
        responses.append(future.result())


In [ ]:
responses

In [ ]:
import os, json
from tqdm import tqdm
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed


def call_openai(entry):
    custom_id = entry["custom_id"]
    prompt    = entry["prompt"]
    model     = custom_id.split("__")[1]
    try:
        resp = client.chat.completions.create(
            model      = model,
            messages   = [{"role":"user","content":prompt}],
            temperature=0.7,
            max_tokens =50
        )
        return {
            "custom_id": custom_id,
            "model":     model,
            "response":  resp.choices[0].message.content.strip()
        }
    except Exception as e:
        return {
            "custom_id": custom_id,
            "model":     model,
            "error":     str(e)
        }

responses = []
# choose a worker count based on your bandwidth & rate‐limit headroom
with ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(call_openai, p) for p in prompts]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Parallelized"):
        responses.append(future.result())

with open("int_data/UK_responses_openai.json", "w", encoding="utf-8") as out:
    json.dump(responses, out, ensure_ascii=False, indent=2)


In [ ]:
responses

# 2. EU HC

In [ ]:
import pandas as pd
import json
import copy

In [ ]:
df = pd.read_csv("int_data/EU_extracted_hc_multilingual.csv")


In [ ]:
df

In [ ]:
import json
from tqdm import tqdm
from concurrent.futures import ThreadPoolExecutor
# Make sure df is already defined (e.g. via pandas.read_csv or similar)
# from openai import OpenAI  <-- imported inside processing functions as needed

# ------------------------------------------------------------------------------
# Dictionaries for language-specific text (as in your original code)
# ------------------------------------------------------------------------------

# -------------------------------------------------------------------------------
# Updated system_instructions: if claim is supported by evidence, answer "1"; if contradicted, answer "0".
# If unsure, choose the option closest to the evidence.
# ------------------------------------------------------------------------------
system_instructions = {
    "English": (
        "You are an expert in evaluating medical claims. Your task is to assess the truthfulness of the provided health claim "
        "using established medical research, guidelines, and evidence. Determine if the claim is true. If the claim is supported by evidence, "
        "respond with '1'. If the claim is contradicted by evidence, respond with '0'. If you are unsure, choose the option that is closest to the available evidence. "
        "Additional context is provided below: the 'Nutrient substance, food or food category' indicates the food or supplement involved, and the 'Health relationship' describes "
        "the intended health effect."
    ),
    "Spanish": (
        "Eres un experto en la evaluación de afirmaciones médicas. Tu tarea es evaluar la veracidad de la afirmación de salud proporcionada "
        "utilizando investigaciones médicas establecidas, directrices y evidencias. Determina si la afirmación es verdadera. Si la afirmación está respaldada por evidencia, "
        "responde con '1'. Si la afirmación contradice la evidencia, responde con '0'. Si no estás seguro, elige la opción que más se aproxime a la evidencia disponible. "
        "Se proporciona contexto adicional a continuación: 'Sustancia nutritiva, alimento o categoría de alimentos' indica el alimento o suplemento involucrado, y 'Relación de salud' describe "
        "el efecto de salud previsto."
    ),
    "Russian": (
        "Вы являетесь экспертом в оценке медицинских заявлений. Ваша задача — оценить достоверность предоставленного заявления о здоровье, "
        "используя установленные медицинские исследования, рекомендации и доказательства. Определите, является ли заявление правдивым. Если заявление подтверждено доказательствами, "
        "ответьте '1'. Если заявление противоречит доказательствам, ответьте '0'. Если вы не уверены, выберите вариант, который наиболее близок к имеющимся доказательствам. "
        "Дополнительный контекст приведён ниже: «Пищевая добавка, продукт или категория продуктов» указывает на используемый продукт или пищевую добавку, а «Связь со здоровьем» описывает "
        "предполагаемый эффект для здоровья."
    ),
    "German": (
        "Sie sind ein Experte für die Bewertung medizinischer Aussagen. Ihre Aufgabe ist es, die Wahrhaftigkeit der vorgelegten Gesundheitsaussage "
        "mithilfe etablierter medizinischer Forschung, Richtlinien und Beweise zu beurteilen. Bestimmen Sie, ob die Aussage wahr ist. Wenn die Aussage durch Beweise gestützt wird, "
        "antworten Sie mit '1'. Wenn die Aussage den Beweisen widerspricht, antworten Sie mit '0'. Falls Sie unsicher sind, wählen Sie bitte die Option, die den verfügbaren Beweisen am nächsten kommt. "
        "Weiterer Kontext wird unten bereitgestellt: 'Nährstoffsubstanz, Lebensmittel oder Lebensmittelkategorie' gibt an, welches Lebensmittel oder Nahrungsergänzungsmittel beteiligt ist, und "
        "'Gesundheitsbeziehung' beschreibt den beabsichtigten gesundheitlichen Effekt."
    ),
    "French": (
        "Vous êtes un expert dans l'évaluation des allégations médicales. Votre tâche consiste à évaluer la véracité de l'allégation de santé fournie "
        "en utilisant des recherches médicales établies, des directives et des preuves. Déterminez si l'allégation est vraie. Si l'allégation est étayée par des preuves, "
        "répondez par '1'. Si l'allégation contredit les preuves, répondez par '0'. Si vous n'êtes pas sûr, choisissez l'option qui se rapproche le plus des preuves disponibles. "
        "Un contexte supplémentaire est fourni ci-dessous : 'Substance nutritive, aliment ou catégorie d'aliments' indique l'aliment ou le complément impliqué, et 'Relation de santé' décrit "
        "l'effet de santé envisagé."
    ),
    "Japanese": (
        "あなたは医療クレームの評価に精通した専門家です。あなたの任務は、確立された医学的研究、ガイドライン、証拠を用いて、提供された健康クレームの真実性を評価することです。 "
        "クレームが真実であるかを判断してください。もし証拠に裏付けられている場合は『1』と回答し、証拠と矛盾する場合は『0』と回答してください。不明な場合は、利用可能な証拠に最も近い選択肢を選んでください。 "
        "以下に追加のコンテキストが提供されています：『栄養素、食品または食品カテゴリー』は関係する食品またはサプリメントを示し、『健康関係』は意図された健康効果を説明します。"
    ),
    "Portuguese": (
        "Você é um especialista na avaliação de reivindicações médicas. Sua tarefa é avaliar a veracidade da reivindicação de saúde fornecida "
        "utilizando pesquisas médicas estabelecidas, diretrizes e evidências. Determine se a reivindicação é verdadeira. Se a reivindicação for suportada por evidências, "
        "responda com '1'. Se a reivindicação contradiz as evidências, responda com '0'. Se estiver em dúvida, escolha a opção que mais se aproxima das evidências disponíveis. "
        "Contexto adicional é fornecido abaixo: 'Substância nutritiva, alimento ou categoria de alimentos' indica o alimento ou suplemento envolvido, e 'Relação de saúde' descreve "
        "o efeito de saúde pretendido."
    ),
    "Turkish": (
        "Tıbbi iddiaları değerlendirme konusunda uzmansınız. Göreviniz, yerleşik tıbbi araştırmalar, yönergeler ve kanıtlar kullanarak "
        "sağlanan sağlık iddiasının doğruluğunu değerlendirmektir. İddianın doğru olup olmadığını belirleyin. İddia kanıtlarla destekleniyorsa, '1' ile cevap verin. "
        "İddia kanıtlara ters düşüyorsa, '0' ile cevap verin. Emin değilseniz, mevcut kanıtlara en yakın olan seçeneği tercih edin. Aşağıda ek bağlam sağlanmıştır: "
        "'Besin maddesi, gıda veya gıda kategorisi' ilgili gıda veya takviyeyi belirtir ve 'Sağlık ilişkisi' amaçlanan sağlık etkisini açıklar."
    ),
    "Italian": (
        "Sei un esperto nella valutazione delle affermazioni mediche. Il tuo compito è valutare la veridicità dell'affermazione sulla salute fornita "
        "utilizzando ricerche mediche consolidate, linee guida ed evidenze. Determina se l'affermazione è vera. Se l'affermazione è supportata dalle evidenze, rispondi con '1'. "
        "Se l'affermazione contraddice le evidenze, rispondi con '0'. Se non sei sicuro, scegli l'opzione che più si avvicina alle evidenze disponibili. "
        "Ulteriori contesti sono forniti di seguito: 'Sostanza nutritiva, alimento o categoria di alimenti' indica l'alimento o il supplemento coinvolto, e 'Relazione di salute' descrive "
        "l'effetto previsto sulla salute."
    ),
    "Persian": (
        "شما یک متخصص در ارزیابی ادعاهای پزشکی هستید. وظیفه شما این است که صحت ادعای ارائه شده در زمینه سلامت را با استفاده از تحقیقات پزشکی معتبر، "
        "دستورالعمل‌ها و شواهد موجود ارزیابی کنید. تعیین کنید که آیا ادعا صحیح است. اگر ادعا با شواهد تأیید شده است، با '1' پاسخ دهید. اگر ادعا با شواهد مغایرت دارد، با '0' پاسخ دهید. "
        "اگر مطمئن نیستید، نزدیک‌ترین گزینه به شواهد موجود را انتخاب کنید. زمینه اضافی در زیر ارائه شده است: 'ماده مغذی، غذا یا دسته‌بندی غذا' نشان‌دهنده غذایی است که در آن ادعا مطرح شده و 'رابطه سلامت' اثر مورد نظر بر سلامت را توضیح می‌دهد."
    ),
    "Dutch": (
        "U bent een expert in het evalueren van medische claims. Uw taak is om de waarheidsgetrouwheid van de verstrekte gezondheidsclaim te beoordelen "
        "met behulp van gevestigde medische onderzoeken, richtlijnen en bewijzen. Bepaal of de claim waar is. Als de claim door bewijzen wordt ondersteund, "
        "antwoord dan met '1'. Als de claim in tegenspraak is met de bewijzen, antwoord dan met '0'. Als u niet zeker bent, kiest u de optie die het dichtst bij het beschikbare bewijs ligt. "
        "Verdere context wordt hieronder gegeven: 'Voedingsstof, voeding of voedselcategorie' geeft aan om welk voedsel of supplement het gaat, en 'Gezondheidsrelatie' beschrijft "
        "het beoogde gezondheidseffect."
    ),
    "Polish": (
        "Jesteś ekspertem w ocenie twierdzeń medycznych. Twoim zadaniem jest ocenić prawdziwość podanego twierdzenia dotyczącego zdrowia, "
        "korzystając z uznanych badań medycznych, wytycznych i dowodów. Określ, czy twierdzenie jest prawdziwe. Jeśli twierdzenie jest poparte dowodami, odpowiedz '1'. "
        "Jeśli twierdzenie przeczy dowodom, odpowiedz '0'. Jeśli nie jesteś pewien, wybierz opcję najbliższą dostępnym dowodom. Dodatkowy kontekst podany jest poniżej: "
        "'Substancja odżywcza, żywność lub kategoria żywności' wskazuje na dany produkt lub suplement, a 'Relacja zdrowotna' opisuje zamierzony efekt zdrowotny."
    ),
    "Chinese": (
        "您是评估医疗声明的专家。您的任务是利用既定的医学研究、指南和证据来评估所提供健康声明的真实性。请判断该声明是否真实。"
        "如果声明得到证据支持，请回答 '1'；如果声明与证据相悖，请回答 '0'；如果不确定，请选择最接近现有证据的选项。"
        "下面提供了额外的上下文：“营养物质、食品或食品类别”指的是所涉及的食品或补充剂，而“健康关系”描述了预期的健康效果。"
    ),
    "Vietnamese": (
        "Bạn là một chuyên gia trong việc đánh giá các tuyên bố y tế. Nhiệm vụ của bạn là đánh giá tính chân thực của tuyên bố về sức khỏe được cung cấp, "
        "dựa trên các nghiên cứu y tế đã được thiết lập, các hướng dẫn và bằng chứng. Hãy xác định xem tuyên bố đó có thật không. Nếu tuyên bố được hỗ trợ bởi bằng chứng, hãy trả lời bằng '1'. "
        "Nếu tuyên bố mâu thuẫn với bằng chứng, hãy trả lời bằng '0'. Nếu không chắc chắn, hãy chọn phương án gần với bằng chứng nhất. "
        "Bối cảnh bổ sung được cung cấp bên dưới: ‘Chất dinh dưỡng, thực phẩm hoặc loại thực phẩm’ chỉ ra sản phẩm hoặc thực phẩm bổ sung liên quan, và ‘Mối quan hệ sức khỏe’ mô tả hiệu ứng sức khỏe dự kiến."
    ),
    "Indonesian": (
        "Anda adalah seorang ahli dalam mengevaluasi klaim medis. Tugas Anda adalah menilai kebenaran klaim kesehatan yang diberikan dengan menggunakan penelitian medis yang mapan, "
        "pedoman, dan bukti. Tentukan apakah klaim tersebut benar. Jika klaim didukung oleh bukti, jawablah dengan '1'. Jika klaim bertentangan dengan bukti, jawablah dengan '0'. "
        "Jika Anda tidak yakin, pilihlah opsi yang paling mendekati bukti yang ada. Konteks tambahan disediakan di bawah ini: 'Zat gizi, makanan atau kategori makanan' menunjukkan makanan atau suplemen yang terlibat, "
        "dan 'Hubungan kesehatan' menggambarkan efek kesehatan yang dimaksudkan."
    ),
    "Czech": (
        "Jste odborník na hodnocení lékařských tvrzení. Vaším úkolem je posoudit pravdivost poskytnutého zdravotního tvrzení pomocí zavedeného lékařského výzkumu, pokynů a důkazů. "
        "Určete, zda je tvrzení pravdivé. Pokud je tvrzení podpořeno důkazy, odpovězte '1'. Pokud tvrzení odporuje důkazům, odpovězte '0'. "
        "Pokud si nejste jisti, vyberte možnost, která se nejvíce blíží dostupným důkazům. Níže je uveden další kontext: 'Výživová látka, potravina nebo kategorie potravin' označuje zapojenou potravinu nebo doplněk stravy a "
        "'Zdravotní vztah' popisuje zamýšlený zdravotní účinek."
    ),
    "Korean": (
        "당신은 의료 주장 평가의 전문가입니다. 확립된 의학적 연구, 지침 및 증거를 활용하여 제공된 건강 주장의 진실성을 평가하십시오. "
        "주장이 증거로 뒷받침되면 '1'로, 증거와 상반되면 '0'로 답하십시오. 만약 확신이 없다면, 가능한 증거에 가장 근접한 선택지를 고르십시오. "
        "아래에 추가 문맥이 제공됩니다: '영양소, 식품 또는 식품 범주'는 관련 식품이나 보충제를 나타내며, '건강 관계'는 의도된 건강 효과를 설명합니다."
    ),
    "Ukrainian": (
        "Ви є експертом з оцінки медичних заяв. Ваше завдання — оцінити достовірність наданої заяви про здоров'я, використовуючи встановлені медичні дослідження, рекомендації та докази. "
        "Визначте, чи є заява правдивою. Якщо заява підтверджена доказами, відповідайте '1'. Якщо заява суперечить доказам, відповідайте '0'. "
        "Якщо ви не впевнені, оберіть варіант, який найбільше відповідає наявним доказам. Додатковий контекст наведено нижче: 'Харчова добавка, їжа або категорія їжі' вказує на залучений продукт або добавку, а "
        "'Зв'язок зі здоров'ям' описує передбачуваний вплив на здоров'я."
    ),
    "Arabic": (
        "أنت خبير في تقييم المطالبات الطبية. مهمتك هي تقييم صحة المطالبة الصحية المقدمة باستخدام البحوث الطبية المعتمدة والإرشادات والأدلة. "
        "حدد ما إذا كانت المطالبة صحيحة. إذا كانت المطالبة مدعومة بالأدلة، فقم بالرد بـ '1'. إذا كانت المطالبة تتناقض مع الأدلة، فقم بالرد بـ '0'. "
        "إذا لم تكن متأكدًا، اختر الخيار الأقرب إلى الأدلة المتوفرة. يتم تقديم سياق إضافي أدناه: 'المادة الغذائية، الطعام أو فئة الطعام' تشير إلى الطعام أو المكمل المعني، و"
        "'العلاقة الصحية' تصف التأثير الصحي المقصود."
    ),
    "Greek": (
        "Είστε ειδικός στην αξιολόγηση ιατρικών ισχυρισμών. Το έργο σας είναι να αξιολογήσετε την αλήθεια του παρεχόμενου ισχυρισμού υγείας, χρησιμοποιώντας καθιερωμένη ιατρική έρευνα, "
        "κατευθυντήριες οδηγίες και αποδείξεις. Καθορίστε αν ο ισχυρισμός είναι αληθής. Εάν ο ισχυρισμός υποστηρίζεται από αποδείξεις, απαντήστε με '1'. Εάν ο ισχυρισμός έρχεται σε αντίθεση με τις αποδείξεις, απαντήστε με '0'. "
        "Εάν δεν είστε βέβαιοι, επιλέξτε την επιλογή που πλησιάζει περισσότερο στις διαθέσιμες αποδείξεις. Επιπλέον πλαίσιο παρέχεται παρακάτω: 'Θρεπτική ουσία, τρόφιμο ή κατηγορία τροφίμων' υποδεικνύει το εμπλεκόμενο τρόφιμο ή συμπλήρωμα, και "
        "'Σχέση υγείας' περιγράφει το επιδιωκόμενο αποτέλεσμα."
    ),
    "Hindi": (
        "आप चिकित्सा दावों का मूल्यांकन करने में विशेषज्ञ हैं। आपका कार्य प्रदान किए गए स्वास्थ्य दावे की सत्यता का आकलन करना है, इसके लिए स्थापित चिकित्सा अनुसंधान, दिशानिर्देश और सबूतों का उपयोग करें। "
        "निर्धारित करें कि दावा सही है। यदि दावा सबूतों द्वारा समर्थित है, तो '1' के साथ उत्तर दें। यदि दावा सबूतों के विपरीत है, तो '0' के साथ उत्तर दें। "
        "यदि आप सुनिश्चित नहीं हैं, तो उपलब्ध सबूतों के अनुसार सबसे उपयुक्त विकल्प चुनें। अतिरिक्त संदर्भ नीचे प्रदान किया गया है: 'पोषक तत्व, खाद्य या खाद्य वर्ग' से तात्पर्य उस खाद्य या अनुपूरक से है जो शामिल है, और "
        "'स्वास्थ्य संबंध' से आशय लक्षित स्वास्थ्य प्रभाव से है।"
    )
}

# ------------------------------------------------------------------------------
# Updated strict_response_lines: only "1" or "0" are allowed, with no extra text.
# ------------------------------------------------------------------------------
strict_response_lines = {
    "English": "Your answer must be exactly one of the following: '1' or '0'. Do not include any additional text.",
    "Spanish": "Su respuesta debe ser exactamente una de las siguientes: '1' o '0'. No incluya ningún texto adicional.",
    "Russian": "Ваш ответ должен быть ровно одним из следующих: '1' или '0'. Не добавляйте никакого дополнительного текста.",
    "German": "Ihre Antwort muss genau eine der folgenden Optionen sein: '1' oder '0'. Fügen Sie keinen zusätzlichen Text hinzu.",
    "French": "Votre réponse doit être exactement l'une des options suivantes : '1' ou '0'. N'incluez aucun texte supplémentaire.",
    "Japanese": "あなたの回答は次のいずれか一つに正確にしてください：『1』または『0』。追加のテキストを含めないでください。",
    "Portuguese": "Sua resposta deve ser exatamente uma das seguintes: '1' ou '0'. Não inclua nenhum texto adicional.",
    "Turkish": "Cevabınız tam olarak aşağıdakilerden biri olmalıdır: '1' veya '0'. Ekstra metin eklemeyin.",
    "Italian": "La tua risposta deve essere esattamente una delle seguenti: '1' o '0'. Non includere testo aggiuntivo.",
    "Persian": "پاسخ شما باید دقیقا یکی از گزینه‌های زیر باشد: '1' یا '0'. هیچ متن اضافی را اضافه نکنید.",
    "Dutch": "Uw antwoord moet exact een van de volgende zijn: '1' of '0'. Voeg geen extra tekst toe.",
    "Polish": "Twoja odpowiedź musi być dokładnie jedną z następujących: '1' lub '0'. Nie dodawaj żadnego dodatkowego tekstu.",
    "Chinese": "您的回答必须严格为以下其中之一：‘1’ 或 ‘0’。请不要包含任何额外的文字。",
    "Vietnamese": "Câu trả lời của bạn phải chính xác là một trong các lựa chọn sau: '1' hoặc '0'. Không thêm bất kỳ văn bản bổ sung nào.",
    "Indonesian": "Jawaban Anda harus tepat salah satu dari yang berikut: '1' atau '0'. Jangan sertakan teks tambahan apa pun.",
    "Czech": "Vaše odpověď musí být přesně jedna z následujících: '1' nebo '0'. Nezahrnujte žádný další text.",
    "Korean": "귀하의 답변은 반드시 다음 중 하나여야 합니다: '1' 또는 '0'. 추가 텍스트를 포함하지 마십시오.",
    "Ukrainian": "Ваша відповідь має бути точно однією з наступних: '1' або '0'. Не додавайте ніякого додаткового тексту.",
    "Arabic": "يجب أن يكون جوابك هو أحد الخيارات التالية بالضبط: '1' أو '0'. لا تقم بإضافة أي نص إضافي.",
    "Greek": "Η απάντησή σας πρέπει να είναι ακριβώς μία από τις εξής: '1' ή '0'. Μην προσθέτετε επιπλέον κείμενο.",
    "Hindi": "आपका उत्तर ठीक उसी में से एक होना चाहिए: '1' या '0'. किसी भी अतिरिक्त पाठ को शामिल न करें।"
}

# ------------------------------------------------------------------------------
# Prompt labels for the claim details in each language
# ------------------------------------------------------------------------------
prompt_labels = {
    "English": {
        "claim": "Claim:",
        "nutrient": "Nutrient substance, food or food category:",
        "health": "Health relationship:"
    },
    "Spanish": {
        "claim": "Afirmación:",
        "nutrient": "Sustancia nutritiva, alimento o categoría de alimentos:",
        "health": "Relación de salud:"
    },
    "Russian": {
        "claim": "Заявление:",
        "nutrient": "Пищевая добавка, продукт или категория продуктов:",
        "health": "Связь со здоровьем:"
    },
    "German": {
        "claim": "Behauptung:",
        "nutrient": "Nährstoffsubstanz, Lebensmittel oder Lebensmittelkategorie:",
        "health": "Gesundheitsbeziehung:"
    },
    "French": {
        "claim": "Allégation:",
        "nutrient": "Substance nutritive, aliment ou catégorie d'aliments:",
        "health": "Relation de santé:"
    },
    "Japanese": {
        "claim": "クレーム:",
        "nutrient": "栄養素、食品または食品カテゴリー:",
        "health": "健康関係:"
    },
    "Portuguese": {
        "claim": "Alegação:",
        "nutrient": "Substância nutritiva, alimento ou categoria de alimentos:",
        "health": "Relação de saúde:"
    },
    "Turkish": {
        "claim": "İddia:",
        "nutrient": "Besin maddesi, gıda veya gıda kategorisi:",
        "health": "Sağlık ilişkisi:"
    },
    "Italian": {
        "claim": "Affermazione:",
        "nutrient": "Sostanza nutritiva, alimento o categoria di alimenti:",
        "health": "Relazione di salute:"
    },
    "Persian": {
        "claim": "ادعا:",
        "nutrient": "ماده مغذی، غذا یا دسته‌بندی غذا:",
        "health": "رابطه سلامت:"
    },
    "Dutch": {
        "claim": "Claim:",
        "nutrient": "Voedingsstof, voeding of voedselcategorie:",
        "health": "Gezondheidsrelatie:"
    },
    "Polish": {
        "claim": "Twierdzenie:",
        "nutrient": "Substancja odżywcza, żywność lub kategoria żywności:",
        "health": "Relacja zdrowotna:"
    },
    "Chinese": {
        "claim": "声明:",
        "nutrient": "营养物质、食品或食品类别:",
        "health": "健康关系:"
    },
    "Vietnamese": {
        "claim": "Tuyên bố:",
        "nutrient": "Chất dinh dưỡng, thực phẩm hoặc loại thực phẩm:",
        "health": "Mối quan hệ sức khỏe:"
    },
    "Indonesian": {
        "claim": "Klaim:",
        "nutrient": "Zat gizi, makanan atau kategori makanan:",
        "health": "Hubungan kesehatan:"
    },
    "Czech": {
        "claim": "Tvrzení:",
        "nutrient": "Výživová látka, potravina nebo kategorie potravin:",
        "health": "Zdravotní vztah:"
    },
    "Korean": {
        "claim": "주장:",
        "nutrient": "영양소, 식품 또는 식품 범주:",
        "health": "건강 관계:"
    },
    "Ukrainian": {
        "claim": "Заява:",
        "nutrient": "Харчова добавка, їжа або категорія їжі:",
        "health": "Зв'язок зі здоров'ям:"
    },
    "Arabic": {
        "claim": "المطالبة:",
        "nutrient": "المادة الغذائية، الطعام أو فئة الطعام:",
        "health": "العلاقة الصحية:"
    },
    "Greek": {
        "claim": "Ισχυρισμός:",
        "nutrient": "Θρεπτική ουσία, τρόφιμο ή κατηγορία τροφίμων:",
        "health": "Σχέση υγείας:"
    },
    "Hindi": {
        "claim": "दावा:",
        "nutrient": "पोषक तत्व, खाद्य या खाद्य वर्ग:",
        "health": "स्वास्थ्य संबंध:"
    }
}

# ------------------------------------------------------------------------------
# Function to create a combined prompt entry for a given claim in a specific language,
# including model and iteration information.
# ------------------------------------------------------------------------------
def create_prompt_entry(claim_id, translation_claim, translation_nutrient_substance, translation_health_relationship, language, model, iteration):
    # Get language-specific texts; default to English if not available
    sys_instr = system_instructions.get(language, system_instructions["English"])
    labels = prompt_labels.get(language, prompt_labels["English"])
    strict_line = strict_response_lines.get(language, strict_response_lines["English"])
    
    # Build the combined prompt:
    # 1. The system instruction in the target language
    # 2. The claim details using the target language's prompt labels
    # 3. The strict instruction in the target language
    prompt_text = (
        f"{sys_instr}\n\n"
        f"{labels['claim']} {translation_claim}\n"
        f"{labels['nutrient']} {translation_nutrient_substance}\n"
        f"{labels['health']} {translation_health_relationship}\n\n"
        f"{strict_line}"
    )
    
    # Create a custom ID that includes the claim ID, language, model, and iteration
    custom_id = f"claim_{claim_id}_{language}__{model}__{iteration}"
    
    return {
        "custom_id": custom_id,
        "prompt": prompt_text
    }

# ------------------------------------------------------------------------------
# Build the list of prompt entries across models and iterations
# ------------------------------------------------------------------------------
prompts = []
models = [
    "gpt-4o-mini-2024-07-18"
]
iterations = range(3)  # e.g., 0, 1, 2

for model in models:
    for _, row in df.iterrows():
        #print(row)
        language = row['language']  # e.g., "English", "Spanish", etc.
        for iteration in iterations:
            entry = create_prompt_entry(
                row['claim_id'],
                row['translation_claim'],
                row['translation_nutrient_substance'],
                row['translation_health_relationship'],
                language,
                model,
                iteration
            )
            prompts.append(entry)


In [ ]:
len(prompts)

In [ ]:
import os
import json
from tqdm import tqdm
from openai import OpenAI
os.environ["OPENAI_API_KEY"]="KEY" # Replace with your actual OpenAI API key

client = OpenAI(api_key=os.environ["OPENAI_API_KEY"])


In [ ]:
import os, json
from tqdm import tqdm
from openai import OpenAI
from concurrent.futures import ThreadPoolExecutor, as_completed


def call_openai(entry):
    custom_id = entry["custom_id"]
    prompt    = entry["prompt"]
    model     = custom_id.split("__")[1]
    try:
        resp = client.chat.completions.create(
            model      = model,
            messages   = [{"role":"user","content":prompt}],
            temperature=0.7,
            max_tokens =50
        )
        return {
            "custom_id": custom_id,
            "model":     model,
            "response":  resp.choices[0].message.content.strip()
        }
    except Exception as e:
        return {
            "custom_id": custom_id,
            "model":     model,
            "error":     str(e)
        }

responses = []
# choose a worker count based on your bandwidth & rate‐limit headroom
with ThreadPoolExecutor(max_workers=50) as executor:
    futures = [executor.submit(call_openai, p) for p in prompts]
    for future in tqdm(as_completed(futures), total=len(futures), desc="Parallelized"):
        responses.append(future.result())

with open("int_data/EU_responses_openai.json", "w", encoding="utf-8") as out:
    json.dump(responses, out, ensure_ascii=False, indent=2)


In [ ]:
responses